<a href="https://colab.research.google.com/github/nachozobian/computer-vision-PyTorch/blob/main/computer_vision_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch computer vision notebook

This notebook is just me taking my first steps with computer vision in PyTorch.

* `torchvision` is part of the PyTorch project for computer vision! 
* `torchvision.datasets` get datasets and data loading functions
* `torchvision.models` get pretrained computer vision models
* `torchvision.transforms` manipulating your vision data to be suitable for ML
* `torch.utils.data.Dataset` Base dataset class for PyTorch
* `torch.utils.data.DataLoader` Python iterable object

In [1]:
#Import PyTorch
import torch
from torch import nn

#Import Torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

#Import matplotlib for visualization
import matplotlib.pyplot as plt